Exploratory analysis

In [7]:
import pandas as pd
import os
thedir = os.getcwd()
datadir = os.path.abspath(os.path.join(os.path.dirname(thedir), '..', 'data/raw'))
readdir = os.path.abspath(os.path.join(os.path.dirname(thedir), '..', 'data/intermediate'))

In [8]:
files = os.listdir(datadir)

In [9]:
files

['Log_204.csv', 'Log_201.csv', 'Log_203.csv', 'Log_202.csv']

In [61]:
mylist = []
mycol = []
for i in files:
    mylist.append(pd.read_csv(F"{datadir}/{i}", header = 1))
    mycol.append(i[:7])

In [62]:
c = mylist[0]
d = mycol[0]

In [64]:
c[d] = d

In [65]:
c.columns

Index(['#', 'Date Time, GMT+02:00',
       'Temp, °C (LGR S/N: 21068247, SEN S/N: 21068247)',
       'Coupler Attached (LGR S/N: 21068247)',
       'Coupler Detached (LGR S/N: 21068247)',
       'Host Connected (LGR S/N: 21068247)', 'Stopped (LGR S/N: 21068247)',
       'End Of File (LGR S/N: 21068247)', 'Log_204'],
      dtype='object')

In [67]:
newcols = [c.columns[0],c.columns[1],c.columns[2]]

In [72]:
df_ = pd.MultiIndex.from_frame(c,)
df_

MultiIndex([(   1, '09/07/22 00:00:00', 18.996,      nan,      nan, ...),
            (   2, '09/07/22 00:10:00', 18.806,      nan,      nan, ...),
            (   3, '09/07/22 00:20:00', 18.616,      nan,      nan, ...),
            (   4, '09/07/22 00:30:00', 18.616,      nan,      nan, ...),
            (   5, '09/07/22 00:40:00', 18.426,      nan,      nan, ...),
            (   6, '09/07/22 00:50:00', 18.426,      nan,      nan, ...),
            (   7, '09/07/22 01:00:00', 18.331,      nan,      nan, ...),
            (   8, '09/07/22 01:10:00',  18.14,      nan,      nan, ...),
            (   9, '09/07/22 01:20:00',  18.14,      nan,      nan, ...),
            (  10, '09/07/22 01:30:00',  17.95,      nan,      nan, ...),
            ...
            (4808, '11/08/22 09:10:00', 19.567,      nan,      nan, ...),
            (4809, '11/08/22 09:20:00', 20.329,      nan,      nan, ...),
            (4810, '11/08/22 09:30:00', 20.901,      nan,      nan, ...),
            (4811, '11

In [ ]:
mylist = []
for i in files:
    a = pd.read_csv(F"{datadir}/{i}",header =1)
    log = i[:7]
    a[log] = log
    a = a.iloc[:-4,[0,1,2,7]]
    mylist.append(a)
pd.concat(a).reset_index().set_index(['index', 'log'])


In [56]:
df = pd.DataFrame()
count = 0
for i in files:
    a = pd.read_csv(F"{datadir}/{i}",header =1)
    logger = i[:7]
    cols = a.columns
    cols = [cols[0],cols[1],cols[2]]
    mlm = zip(logger,cols)
for i in mlm:
    print(i)

('L', '#')
('o', 'Date Time, GMT+02:00')
('g', 'Temp, °C (LGR S/N: 20350621, SEN S/N: 20350621)')


In [44]:
for i in files.index:
    

SyntaxError: incomplete input (524530521.py, line 2)